In [21]:
import pandas as pd
import numpy as np

from itertools import combinations
from scipy.stats import pearsonr
from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests 

In [4]:
df = pd.read_csv('AUCs.txt', sep = '\t')

In [11]:
df.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. 

Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.



In [23]:
ddf = pd.DataFrame(columns = ['Classifier 1', 'Classifier 2', 'Wilx', 'p-value'])
n = 0 
for i,j in combinations(np.arange(1,5), 2):
    ddf.loc[n, 'Classifier 1'], ddf.loc[n, 'Classifier 2'] = df.columns[i], df.columns[j]
    ddf.loc[n, 'Wilx'] = stats.wilcoxon(df.iloc[:,i], df.iloc[:,j])[0]
    ddf.loc[n, 'p-value'] = stats.wilcoxon(df.iloc[:,i], df.iloc[:,j])[1]
    n += 1

In [50]:
ddf

,Classifier 1,Classifier 2,Wilx,p-value
0,C4.5,C4.5+m,6.5,0.0107571
1,C4.5,C4.5+cf,43,0.861262
2,C4.5,C4.5+m+cf,11,0.0159064
3,C4.5+m,C4.5+cf,17,0.0463327
4,C4.5+m,C4.5+m+cf,22,0.327826
5,C4.5+cf,C4.5+m+cf,10,0.0229091


Сколько статистически значимых на уровне 0.05 различий мы обнаружили?



In [27]:
ddf[ddf['p-value'] <= 0.05].shape

(4, 4)

Судя по данным из предыдущего опроса, настройка какого из параметров классификатора даёт более значимое увеличение качества?



In [45]:
df.describe()

,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
count,14.000000,14.000000,14.000000,14.000000
mean,0.804929,0.820429,0.808786,0.827214
std,0.160187,0.158583,0.167566,0.154548
min,0.583000,0.583000,0.563000,0.569000
25%,0.636250,0.666500,0.624000,0.673000
50%,0.828500,0.863000,0.876000,0.886500
75%,0.950500,0.968750,0.960250,0.965750
max,1.000000,1.000000,1.000000,1.000000


Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?



In [48]:
multipletests(ddf['p-value'], alpha=0.05,method = 'holm')

(array([False, False, False, False, False, False]),
 array([0.06454279871873778, 0.861262330095348, 0.07953222050851687,
        0.13899818938018618, 0.6556513516892812, 0.09163639741742635],
       dtype=object),
 0.008512444610847103,
 0.008333333333333333)

Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?



In [49]:
multipletests(ddf['p-value'], alpha=0.05,method = 'fdr_bh')

(array([ True, False,  True, False, False,  True]),
 array([0.045818198708713176, 0.861262330095348, 0.045818198708713176,
        0.06949909469009309, 0.3933908110135687, 0.045818198708713176],
       dtype=object),
 0.008512444610847103,
 0.008333333333333333)